In [4]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from sklearn.model_selection import train_test_split

# Constants
IMAGE_SIZE = (224, 224)
BATCH_SIZE = 32
NUM_CLASSES = 2  # 'Number Plate' and 'Not a Number Plate'

# Data Preprocessing
def preprocess_data(data_dir):
    images = []
    labels = []

    for folder_name in os.listdir(data_dir):
        folder_path = os.path.join(data_dir, folder_name)
        label = 0 if folder_name == 'non_number_plate' else 1

        for filename in os.listdir(folder_path):
            img_path = os.path.join(folder_path, filename)
            img = cv2.imread(img_path)
            img = cv2.resize(img, IMAGE_SIZE)
            img = preprocess_input(img)
            images.append(img)
            labels.append(label)

    return np.array(images), np.array(labels)

# Load and preprocess the dataset
data_directory = 'indian_license_plate.xml'
X, y = preprocess_data(data_directory)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build a pre-trained MobileNetV2 model
base_model = MobileNetV2(input_shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3),
                          include_top=False,
                          weights='imagenet')
base_model.trainable = False

model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation='relu'),
    layers.Dense(NUM_CLASSES, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

# Create an image data generator for training and validation
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(rotation_range=20,
                                                               width_shift_range=0.2,
                                                               height_shift_range=0.2,
                                                               horizontal_flip=True,
                                                               validation_split=0.2)
train_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE, subset='training')
val_generator = train_datagen.flow(X_train, y_train, batch_size=BATCH_SIZE, subset='validation')

# Train the model
history = model.fit(train_generator,
                    epochs=10,
                    validation_data=val_generator)

# Evaluate the model
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test accuracy: {test_acc * 100:.2f}%')

# Save the model
model.save('license_plate_detection_model.h5')


NotADirectoryError: [WinError 267] The directory name is invalid: 'indian_license_plate.xml'